# 🚀 Your First AI Agent: From Prompt to Action

**Welcome to the Kaggle 5-day Agents course!**

This notebook is your first step into building AI agents. An agent can do more than just respond to a prompt — it can **take actions** to find information or get things done.

In this notebook, you'll:

- ✅ Install [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Configure your API key to use the Gemini model
- ✅ Build your first simple agent
- ✅ Run your agent and watch it use a tool (like Google Search) to answer a question

## ‼️ Please Read

> ❌ **ℹ️ Note: No submission required!**
> This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

> ⏸️ **Note:**  When you first start the notebook via running a cell you might see a banner in the notebook header that reads **"Waiting for the next available notebook"**. The queue should drop rapidly; however, during peak bursts you might have to wait a few minutes.

> ❌ **Note:** Avoid using the **Run all** cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order - one at a time. [See FAQ on 429 errors for more information.](https://www.kaggle.com/code/kaggle5daysofai/day-0-troubleshooting-and-faqs)

**For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.**

## 📖 Get started with Kaggle Notebooks

If this is your first time using Kaggle Notebooks, welcome! You can learn more about using Kaggle Notebooks [in the documentation](https://www.kaggle.com/docs/notebooks).

Here's how to get started:

**1. Verify Your Account (Required)**

To use the Kaggle Notebooks in this course, you'll need to verify your account with a phone number.

You can do this in your [Kaggle settings](https://www.kaggle.com/settings).

**2. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

This creates a private copy of the notebook just for you.

**3. Run Code Cells**

Once you have your copy, you can run code.

Click the ▶️ Run button next to any code cell to execute it.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

Run the cells in order from top to bottom.

**4. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.

## ⚙️ Section 1: Setup

### 1.1: Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

In [2]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.8 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,

### 1.2: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [4]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


### 1.5: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

---

## 🤖 Section 2: Your first AI Agent with ADK

### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

### 2.2 Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [7]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


### 2.3 Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [8]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK](https://google.github.io/adk-docs/runtime/).

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [9]:
response = await runner.run_debug(
    "How to correlate security operations with communism"
)


 ### Created new session: debug_session_id

User > How to correlate security operations with communism
helpful_assistant > While the terms "security operations" and "communism" might seem disparate, historical and theoretical connections can be drawn, primarily through the lens of the Cold War and ideological conflict.

During the Cold War, security operations were deeply intertwined with the global struggle between communist and capitalist ideologies. The Soviet Union and its allies employed extensive security apparatuses, including intelligence agencies like the KGB, to maintain internal control, suppress dissent, and conduct espionage against Western powers. These operations were often justified by the need to protect the communist state from internal and external threats, including perceived "international communist conspiracies".

Conversely, Western nations, particularly the United States, developed robust security operations to counter the spread of communism. This included the

### Created new session: debug_session_id

User > How to correlate security operations with communism
helpful_assistant > A direct correlation between "security operations" and "communism" can be observed through historical events and the nature of communist states. This correlation primarily stems from the emphasis on state control, ideology, and the perceived need to protect the communist system from internal and external threats.

Here are several ways in which security operations and communism are intertwined:

*   **State Surveillance and Control:** Communist regimes, by their nature, often prioritize a strong state apparatus to maintain power and enforce ideology. This translates into extensive surveillance operations, aiming to monitor citizens, suppress dissent, and identify potential "enemies of the state." The existence of secret police forces and intelligence agencies was a hallmark of many communist states, directly linked to internal security operations. For instance, the Soviet Union utilized various intelligence agencies like the GRU, OGPU, NKVD, and KGB for espionage and internal security.
*   **Ideological Security:** In communist systems, ideology is paramount and is considered an integral component of national security. Security operations are often geared towards protecting this ideology from "corrupting" external influences and internal deviations. This can involve censorship, propaganda, and the suppression of ideas deemed counter-revolutionary. The term "ideological security" itself suggests a security apparatus focused on safeguarding the communist party's doctrines and rule.
*   **Suppression of Dissent and Opposition:** Communist parties, in their pursuit of a classless society, often view any form of opposition as a threat to their revolution and the state. Security operations, therefore, are crucial for identifying, neutralizing, and often persecuting political dissidents, "saboteurs," and "foreign agents". This can range from intelligence gathering and surveillance to more overt actions like arrests, imprisonment, and even violence.
*   **"Active Measures" and Espionage:** Communist states, particularly the Soviet Union, employed "active measures" as a form of political warfare. These encompassed a wide range of operations, including propaganda, disinformation, and even terrorist acts, all aimed at advancing communist interests and undermining adversaries. Espionage activities, such as the transmission of confidential information (like atomic bomb secrets), were also a significant component of these security operations.
*   **Cold War Context:** The Cold War epitomized the security operations associated with communism. The ideological struggle between communist and capitalist blocs led to massive investments in intelligence, counter-intelligence, and military operations. Both sides engaged in covert operations, espionage, and proxy conflicts, with communist states like the Soviet Union actively intervening to maintain communist rule in Eastern Europe and elsewhere.
*   **Terrorism and Insurgency:** In some instances, communist groups have engaged in terrorism and guerrilla warfare to achieve their political objectives, often with state backing. This further blurs the lines between security operations and acts of political violence conducted in the name of communism.

In essence, the security operations within communist states were not merely about traditional defense but were deeply embedded in the ideology and political system, serving to preserve the party's power, enforce ideological conformity, and combat perceived threats to the communist order

You can see a summary of ADK and its available languages in the response.

### 2.4 How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

### 🚀 2.5 Your Turn!

This is your chance to see the agent in action. Ask it a question that requires current information.

Try one of these, or make up your own:

- What's the weather in London?
- Who won the last soccer world cup?
- What new movies are showing in theaters now?

In [10]:
response = await runner.run_debug("What's the top 5 cyber security breach in today's tech news")


 ### Continue session: debug_session_id

User > What's the top 5 cyber security breach in today's tech news
helpful_assistant > It appears you're looking for recent, high-profile cybersecurity breaches making headlines. Based on current tech news, here are five significant incidents:

1.  **Habib Bank AG Zurich Hit by Qilin Ransomware:** In early November 2025, the Qilin ransomware group claimed to have stolen over 2.5 terabytes of data from Habib Bank AG Zurich. This alleged theft includes customer details, transaction records, and internal source code, potentially exposing millions of files. The bank, which has a global presence, has not yet confirmed the breach, but the incident highlights the ongoing threat ransomware poses to financial institutions.

2.  **Conduent Business Services Data Breach:** This breach, which occurred in late 2024 and early 2025, impacted nearly 4.3 million individuals. Conduent, a business services provider, had its systems compromised, leading to the exp

### Continue session: debug_session_id

User > What's the top 5 cyber security breach in today's tech news
helpful_assistant > Given that "today's tech news" is a rapidly changing landscape, pinpointing the absolute "top 5" can be challenging as new breaches are reported frequently. However, based on recent reports and their significant impact, here are five prominent cybersecurity breaches that have been making headlines:

1.  **Habib Bank AG Zurich and Qilin Ransomware Group:** The Qilin ransomware group has claimed responsibility for an attack on Habib Bank AG Zurich, alleging the theft of over 2.5 terabytes of data, including customer details and internal source code. This incident highlights the ongoing threat of ransomware to global banking institutions.

2.  **TransUnion Data Breach:** Credit reporting agency TransUnion experienced a significant data breach that exposed the personal information of over 4.4 million individuals. The breach, linked to a third-party application, compromised names, dates of birth, Social Security numbers, and other sensitive data.

3.  **Volvo Group Data Breach:** Volvo Group confirmed a major data breach resulting from a ransomware attack on its Swedish HR software provider, Miljödata. The incident, attributed to the DataCarry ransomware group, exposed sensitive employee data, including Social Security Numbers for some U.S. staff.

4.  **Orange Telecom Data Breach:** French telecommunications giant Orange SA reported a ransomware attack that led to the theft and publication of business customer data on the dark web. While Orange stated the access was limited and the data was outdated or of low sensitivity, the incident underscores the risks faced by large telecommunication companies.

5.  **National Defense Corporation (NDC) and Interlock Ransomware Group:** The Interlock Ransomware Group targeted NDC and its subsidiary AMTEC, exfiltrating 4.2 terabytes of data that was later leaked on the dark web. NDC, which manufactures ammunition, reported the breach as a "system outage caused by a cybersecurity incident," indicating a potential national security implication.

These breaches represent a range of attack vectors and impacts, from ransomware and third-party vulnerabilities to large-scale data theft, underscoring the persistent and evolving nature of cyber threats in the current tech landscape.

---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

In [11]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



Get your custom URL to access the ADK web UI in the Kaggle Notebooks environment:

In [14]:
url_prefix = get_adk_proxy_url()

Now we can run ADK web:

In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [101]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http:

Now you can access the ADK dev UI using the link above.

Once you open the link, you'll see the ADK web interface where you can ask your ADK agent questions.

Note: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.

‼️ **IMPORTANT: DO NOT SHARE THE PROXY LINK** with anyone - treat it as sensitive data as it contains your authentication token in the URL.

---

## ✅ Congratulations!

You've built and run your first agent with ADK! You've just seen the core concept of agent development in action.

The big takeaway is that your agent didn't just *respond*—it **reasoned** that it needed more information and then **acted** by using a tool. This ability to take action is the foundation of all agent-based AI.

**ℹ️ Note: No submission required!**

This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

Refer to the following documentation to learn more:

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK Quickstart for Python](https://google.github.io/adk-docs/get-started/python/)
- [ADK Agents Overview](https://google.github.io/adk-docs/agents/)
- [ADK Tools Overview](https://google.github.io/adk-docs/tools/)

### 🎯 Next Steps

Ready for the next challenge? Continue to the next notebook to learn how to **architect multi-agent systems.**